# Hello World Using Flask

In [78]:
import os
hello_world_script_file = os.path.join(os.path.pardir, 'src', 'models', 'hello_world_api.py')

In [79]:
%%writefile $hello_world_script_file

from flask import Flask, request

app = Flask(__name__)

@app.route('/api', methods = ['POST'])
def say_hello():
    data = request.get_json(force = True)
    name = data['name']
    return "hello {0}".format(name)

if __name__ == '__main__':
    app.run(port = 10001, debug = True)

Overwriting ../src/models/hello_world_api.py


In [80]:
import json
import requests

In [81]:
url = 'http://127.0.0.1:10001/api'
data = json.dumps({'name' : 'Zach'})
r = requests.post(url, data)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=10001): Max retries exceeded with url: /api (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11c276f50>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
print(r.text)

# Machine Learning API Using Flask

## Building the API

In [82]:
machine_learning_api_script_file = os.path.join(os.path.pardir, 'src', 'models', 'machine_learning_api.py')

In [83]:
%%writefile $machine_learning_api_script_file

from flask import Flask, request
import pandas as pd
import numpy as np
import json
import pickle
import os

app = Flask(__name__)

# Load Model and Scaler Files
model_path = os.path.join(os.path.pardir, os.path.pardir, 'models')
model_filepath = os.path.join(model_path, 'lr_model.pkl')
scaler_filepath = os.path.join(model_path, 'lr_scaler.pkl')

scaler = pickle.load(open(scaler_filepath, 'rb'))
model = pickle.load(open(model_filepath, 'rb'))

# columns
columns = [u'Age', u'Fare', u'FamilySize',
       u'IsMother', u'IsMale', u'Deck_A', u'Deck_B', u'Deck_C', u'Deck_D',
       u'Deck_E', u'Deck_F', u'Deck_G', u'Deck_Z', u'Pclass_1', u'Pclass_2',
       u'Pclass_3', u'Title_Lady', u'Title_Master', u'Title_Miss', u'Title_Mr',
       u'Title_Mrs', u'Title_Officer', u'Title_Sir', u'Fare_Bin_very_low',
       u'Fare_Bin_low', u'Fare_Bin_high', u'Fare_Bin_very_high', u'Embarked_C',
       u'Embarked_Q', u'Embarked_S', u'AgeState_Adult', u'AgeState_Child'] 

@app.route('/api', methods=['POST'])
def make_prediction():
    # read json object and conver to json string
    data = json.dumps(request.get_json(force=True))
    # create pandas dataframe using json string
    df = pd.read_json(data)
    # extract passengerIds
    passenger_ids = df['PassengerId'].ravel()
    # actual survived values
    actuals = df['Survived'].ravel()
    # extract required columns based and convert to matrix
    X = df[columns].as_matrix().astype('float')
    # transform the input
    X_scaled = scaler.transform(X)
    # make predictions
    predictions = model.predict(X_scaled)
    # create response dataframe
    df_response = pd.DataFrame({'PassengerId': passenger_ids, 'Predicted' : predictions, 'Actual' : actuals})
    # return json 
    return df_response.to_json()
 

if __name__ == '__main__':
    # host flask app at port 10001
    app.run(port=10001, debug=True)

Overwriting ../src/models/machine_learning_api.py


## Invoking API Using Requests

In [128]:
import os
import pandas as pd
processed_data_path = os.path.join(os.path.pardir,'data','processed')
train_file_path = os.path.join(processed_data_path, 'train.csv')
train_df = pd.read_csv(train_file_path)

In [129]:
survived_passengers = train_df[train_df['Survived'] == 1][:5]

In [130]:
survived_passengers

,PassengerId,Survived,Age,Fare,FamilySize,IsMother,IsMale,Deck_A,Deck_B,Deck_C,...,Title_Sir,Fare_Bin_very_low,Fare_Bin_low,Fare_Bin_high,Fare_Bin_very_high,Embarked_C,Embarked_Q,Embarked_S,AgeState_Adult,AgeState_Child
1,2,1,38.0,71.2833,2,0,0,False,False,True,...,False,False,False,False,True,True,False,False,True,False
2,3,1,26.0,7.9250,1,0,0,False,False,False,...,False,False,True,False,False,False,False,True,True,False
3,4,1,35.0,53.1000,2,0,0,False,False,True,...,False,False,False,False,True,False,False,True,True,False
8,9,1,27.0,11.1333,3,1,0,False,False,False,...,False,False,True,False,False,False,False,True,True,False
9,10,1,14.0,30.0708,2,0,0,False,False,False,...,False,False,False,True,False,True,False,False,False,True


In [140]:
import requests
import json
def make_api_request(data):
    # url for api
    url = 'http://127.0.0.1:10001/api'
    # make post request
    r = requests.post(url, data)
    #return r.text
    return r.json()
    

In [133]:
survived_passengers.to_json()

'{"PassengerId":{"1":2,"2":3,"3":4,"8":9,"9":10},"Survived":{"1":1,"2":1,"3":1,"8":1,"9":1},"Age":{"1":38.0,"2":26.0,"3":35.0,"8":27.0,"9":14.0},"Fare":{"1":71.2833,"2":7.925,"3":53.1,"8":11.1333,"9":30.0708},"FamilySize":{"1":2,"2":1,"3":2,"8":3,"9":2},"IsMother":{"1":0,"2":0,"3":0,"8":1,"9":0},"IsMale":{"1":0,"2":0,"3":0,"8":0,"9":0},"Deck_A":{"1":false,"2":false,"3":false,"8":false,"9":false},"Deck_B":{"1":false,"2":false,"3":false,"8":false,"9":false},"Deck_C":{"1":true,"2":false,"3":true,"8":false,"9":false},"Deck_D":{"1":false,"2":false,"3":false,"8":false,"9":false},"Deck_E":{"1":false,"2":false,"3":false,"8":false,"9":false},"Deck_F":{"1":false,"2":false,"3":false,"8":false,"9":false},"Deck_G":{"1":false,"2":false,"3":false,"8":false,"9":false},"Deck_Z":{"1":false,"2":true,"3":false,"8":true,"9":true},"Pclass_1":{"1":true,"2":false,"3":true,"8":false,"9":false},"Pclass_2":{"1":false,"2":false,"3":false,"8":false,"9":true},"Pclass_3":{"1":false,"2":true,"3":false,"8":true,"9":fa

In [139]:
make_api_request(json.loads(survived_passengers.to_json()))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)